<a href="https://colab.research.google.com/github/Chun-Ping-Wang/data-course-sample/blob/main/homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-08 11:52:30--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.25’

All_Beauty.csv.25   100%[===================>]  14.78M  64.3MB/s    in 0.2s    

2022-01-08 11:52:30 (64.3 MB/s) - ‘All_Beauty.csv.25’ saved [15499476/15499476]

--2022-01-08 11:52:31--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.25’

meta_All_Beauty.jso 100%[===================>]   9.85M  59.6MB/s    in 0.2s

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01') &
    (ratings['DATE'] >= '2017-03-01')
]
#過濾重複資料：如果重複的話，只留下最新的一筆評分。
print(ratings_trainings)
ratings_trainings = ratings_trainings.sort_values("DATE", ascending=False).groupby(['reviewerID', 'asin']).head(1)
print(ratings_trainings)

ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

              asin      reviewerID  overall  unixReviewTime       DATE
7       014789302X  A26PO1B2Q2G1CS      1.0      1491782400 2017-04-10
32      1620213982   A9KCT397IWK97      5.0      1525737600 2018-05-08
33      1620213982  A36Z95ZFDMNL99      5.0      1525737600 2018-05-08
34      1620213982   AB6RYXLZSKHIR      5.0      1525564800 2018-05-06
35      1620213982  A210V878QFZW9V      5.0      1525305600 2018-05-03
...            ...             ...      ...             ...        ...
371338  B01HJEGTYK  A338U2R53FLF9U      5.0      1509148800 2017-10-28
371339  B01HJEGTYK  A3EN87RJFCIORB      4.0      1500681600 2017-07-22
371340  B01HJEGTYK  A202DCI7TV1022      1.0      1500508800 2017-07-20
371341  B01HJEGTYK  A3FSOR5IJOFIBE      5.0      1489622400 2017-03-16
371342  B01HJEGTYK  A1B5DK6CTP2P24      5.0      1488326400 2017-03-01

[84656 rows x 5 columns]
              asin      reviewerID  overall  unixReviewTime       DATE
240477  B01DKQAXC0  A2IW9URACR5LUD      4.0      15

## 產生推薦(cf-used-based)

In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

# header: user_id,item_id,rating,timestamp

def recommender_cfu(training_data, users=[], k=10):

    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    user_to_items = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])

        user_to_items[user][item] = rating

    print("total users before filtering: ", len(user_to_items))

    # remove obscure user to decrease data size
    # filtering params
    remove_obscure_user = True
    user_rating_threshold = 3
    all_users = list(user_to_items.keys())
    for user in all_users:
        ratings = user_to_items[user]
        if remove_obscure_user and len(ratings) < user_rating_threshold:
            del user_to_items[user]

    print("total users  after filtering: ", len(user_to_items))

    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
        for item, rating in items.items():
            item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    print('init_sim')
    print()
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
        if len(user_ratings) > 1:
            # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
            for user1, user2 in combinations(user_ratings.keys(), 2):
                xy = user_ratings[user1] * user_ratings[user2]
                xx = user_ratings[user1] ** 2
                yy = user_ratings[user2] ** 2
                pre_user_similarity[user1][user2][0] += xy
                pre_user_similarity[user1][user2][1] += xx
                pre_user_similarity[user1][user2][2] += yy

                pre_user_similarity[user2][user1][0] += xy
                pre_user_similarity[user2][user1][1] += xx
                pre_user_similarity[user2][user1][2] += yy
        index += 1

    user_similarity = {}
    for src_user in pre_user_similarity:
        user_similarity_order = []
        for dst_user, val in pre_user_similarity[src_user].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(user_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    user_similarity_order.insert(i, (dst_user, similarity))
                    break
            else:
                user_similarity_order.append((dst_user, similarity))
        user_similarity[src_user] = user_similarity_order

    recommendation = {}
    for user in users:
        if user in user_similarity:
            sim_users = user_similarity[user]
            recommended_items = []
            recommended_items_set = set()
            user_have_rated = set(user_to_items[user])
            stop_recommend = False
            for sim_user, _ in sim_users:
                items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
                for item, _ in items_from_sim_user:
                    if item not in user_have_rated and item not in recommended_items_set:
                        recommended_items.append(item)
                        recommended_items_set.add(item)
                    if len(recommended_items) >= k:
                        stop_recommend = True
                        break
                if stop_recommend:
                    break
            recommendation[user] = recommended_items
        else:
            recommendation[user] = []
    return recommendation

## 產生推薦(cf-item-based)

In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

def recommender_cfi(training_data, users=[], k=10):
    # print(training_data)
  
    # print(training_data.shape())

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    print("data converted")

    

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        recommendation[user] = items
    return recommendation    

## 產生推薦(cf-surprise-knn)

In [ ]:
from collections import defaultdict


def get_top_n(predictions, n=10):
    # 首先將預測資料對應到 user
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # 將預測值進行排序，並且選取最高的 n 個值回傳
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n


def recommend_items(model, user, items_user_not_rated, ratings_mean, k):
    # 取出測試資料
    user_recommend_data = [(user, item, ratings_mean) for item in items_user_not_rated]

    # 預測
    predictions = model.test(user_recommend_data)
    top_n = get_top_n(predictions, n=k)
    return [top[0] for top in top_n[user]]

import time
import pandas as pd
!pip install surprise
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic

def recommender_knn(training_data, users=[], k=30, user_based=False, algo=KNNBasic):

    training_data = (
        training_data
        .sort_values("DATE", ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )

    reader = Reader(rating_scale=(0, 5))
    training_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(training_data, reader=reader)

    sim_options = {
        'name': 'cosine',
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)

    recommendation = {}
    for user in users:
        items_user_rated = set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        recommendation[user] = recommend_item_list

    return recommendation

## 選擇推薦法

In [ ]:
# cf_used_based = recommender_cfu(ratings_trainings, users) #score:0
# cf_item_based = recommender_cfi(ratings_trainings, users) #score:0.0016
cf_surprise_knn = recommender_knn(ratings_trainings, users) 
#score:k10 0.0016 / k20 0.0033 / k30 0.0067 無篩選日期但colab跑不動
#score:k10 0.0016 / k20 0.0016 / k30 0.0016 篩一年半內
#基於k30 score: KNN_k=default 0.0016 / KNN_k=31 0.0016 / KNN_k=21 0.0016 /  KNN_k=11 0.0016
 


ratings_by_user = cf_surprise_knn
ratings_by_user

Computing the cosine similarity matrix...
Done computing similarity matrix.


{'A100XQFWKQ30O2': [],
 'A103T1QOGFCSEH': [],
 'A106UKKSJ2KXPF': [],
 'A10A7GV4D5A11V': [],
 'A1119JJ37ZLB8R': [],
 'A113UOOLBSZN52': [],
 'A12M4U7WK4ALCR': [],
 'A12T8YTW6VWT7S': [],
 'A1364JXGKB46MM': [],
 'A137DALOQFKBTI': [],
 'A13FEZ3WV7S2EY': [],
 'A13IV4I1B0RXMG': [],
 'A13JU88JAHN72I': [],
 'A13K55R6VH1OOD': [],
 'A13P7VFU075A': [],
 'A13SWYE4QLB6NG': [],
 'A13ZTQ0Q4ATA41': [],
 'A142EDN04OD62U': [],
 'A142I22FIC8MZK': [],
 'A14834QTII5TLT': [],
 'A14A447VPACTBC': [],
 'A14AP6MN5XO6LB': [],
 'A14CLF25IX25US': [],
 'A14LYXC3HTBAHI': [],
 'A14VUW4KZ34EOE': [],
 'A14Y32P26G9YL': [],
 'A157T25PBS7MX4': [],
 'A15HZDSERD85C8': [],
 'A15JJ8J1FGADIX': [],
 'A15ZCL70JXXH89': [],
 'A1617KN2IAWZ6J': [],
 'A16E0O88262HKA': [],
 'A16NSZ58PTVIYF': [],
 'A16UGDXRTDLJG5': [],
 'A16X9HR3UFQQXY': [],
 'A16Y7V1CZCWKFV': ['B017BJ8KFE',
  'B01DKI8S1Q',
  'B017CHORY8',
  'B00X3T6FEU',
  'B01ENSF702',
  'B00JYGWWIO',
  'B00NV6DMG4',
  'B006IB5T4W',
  'B00L88S190',
  'B000FVHRXC',
  'B01C3UL1ZM',
  'B

## 加入Rule Based

In [ ]:
import numpy as np
#資料整理
#metadata['rank'] to int
metadata['rank'] = metadata['rank'].str.split(expand=True)[0]
metadata['rank'] = metadata['rank'].str.replace(',', '')
metadata['rank']  = pd.to_numeric(metadata['rank'], errors='coerce').astype(np.int64,errors='ignore')
#metadata['price'] to int
metadata['price'] = metadata['price'].str.replace(',', '')
metadata['price'] = metadata['price'].str.replace('$', '')
metadata['price'] = pd.to_numeric(metadata['price'], errors='coerce').astype(np.int64,errors='ignore')

metadata = metadata.replace(np.nan, 0)

#filter1評論時間 1年/6m/3m/2m: 2m表現最好
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01') &
    (ratings['DATE'] >= '2018-07-01')
]
ratings_trainings = ratings_trainings[['asin','reviewerID','overall']]
#ratings_trainings add ['COUNT']
gp_count = ratings_trainings.groupby('asin').count().reset_index()
gp_count['COUNT'] =  gp_count['reviewerID']
gp_count = gp_count[['asin','COUNT']]
ratings_trainings = ratings_trainings.set_index('asin').join(gp_count.set_index('asin'))
ratings_trainings = ratings_trainings.reset_index()
ratings_trainings['COUNT'] = pd.to_numeric(ratings_trainings['COUNT'], errors='coerce').astype(np.int64,errors='ignore')
print(ratings_trainings)

###產生推薦
product_df = metadata[['asin','rank','price','also_view']]
mix_df = ratings_trainings.set_index('asin').join(product_df.set_index('asin')) 
mean_df = mix_df.groupby('asin').mean().reset_index() #groupby asin, mean overall and rank
# mean_df = mean_df[mean_df['rank'] < 100000 ] 不採用
# mean_df = mean_df[mean_df['price'] < 30 ] 不採用，嘗試過各種級距的price但無幫助(EDA中)
print(mean_df)

also_v = ratings_trainings[['asin', 'reviewerID']]
also_view = also_v.set_index('asin').join(product_df.set_index('asin')) 
also_view = also_view.reset_index()[['reviewerID','also_view']]
also_view_df = also_view.drop_duplicates(subset='reviewerID', keep=False, ignore_index=True)
print(also_view_df)

#滿意度3/4/4.5/4.8 4表現最好 按評論數降序排列
rb_df = mean_df[mean_df.overall >= 4].sort_values('COUNT', ascending=False).head(10)
rb_list = rb_df['asin'].tolist()
print(rb_df)

#將KNN return為[]的值用rule based填入 
for key, value in ratings_by_user.items():
  if value == []:
    ratings_by_user[key] = rb_list

ratings_by_user

            asin      reviewerID  overall  COUNT
0     B000FOI48G  A2QT2OB69RY8E2      5.0      6
1     B000FOI48G    AS9SS0RS2CHI      5.0      6
2     B000FOI48G  A1DQ9NGS8MQOVC      5.0      6
3     B000FOI48G  A1LXN3903CZLYO      5.0      6
4     B000FOI48G  A22FXBTMFYU6LD      5.0      6
...          ...             ...      ...    ...
4044  B01HGJHL8G   AOXZ0K779O1KK      5.0      4
4045  B01HHWQX8G  A30NK6QIBSY77I      3.0      2
4046  B01HHWQX8G  A3A91XN9GD4NPR      5.0      2
4047  B01HI1YKIG  A2NUTIW490KHMC      4.0      2
4048  B01HI1YKIG  A1NBOIYBASRIO2      5.0      2

[4049 rows x 4 columns]
            asin  overall  COUNT       rank  price
0     B000FOI48G     5.00      6    19437.0   0.00
1     B000HLCMN0     5.00      1  1438033.0   5.20
2     B000HRRBLC     5.00      1   318902.0  57.59
3     B000MAJD4W     5.00      1   479011.0   0.00
4     B000VH6YM0     5.00      1   314819.0   5.60
...          ...      ...    ...        ...    ...
1654  B01HEYYRIK     3.00     

{'A100XQFWKQ30O2': ['B00W259T7G',
  'B013XKHA4M',
  'B0195R1FT8',
  'B01DLR9IDI',
  'B0168SXRR0',
  'B00X4DKZKU',
  'B00Q794RMA',
  'B01ENS6XDC',
  'B01C39X6TW',
  'B0189NOD4I'],
 'A103T1QOGFCSEH': ['B00W259T7G',
  'B013XKHA4M',
  'B0195R1FT8',
  'B01DLR9IDI',
  'B0168SXRR0',
  'B00X4DKZKU',
  'B00Q794RMA',
  'B01ENS6XDC',
  'B01C39X6TW',
  'B0189NOD4I'],
 'A106UKKSJ2KXPF': ['B00W259T7G',
  'B013XKHA4M',
  'B0195R1FT8',
  'B01DLR9IDI',
  'B0168SXRR0',
  'B00X4DKZKU',
  'B00Q794RMA',
  'B01ENS6XDC',
  'B01C39X6TW',
  'B0189NOD4I'],
 'A10A7GV4D5A11V': ['B00W259T7G',
  'B013XKHA4M',
  'B0195R1FT8',
  'B01DLR9IDI',
  'B0168SXRR0',
  'B00X4DKZKU',
  'B00Q794RMA',
  'B01ENS6XDC',
  'B01C39X6TW',
  'B0189NOD4I'],
 'A1119JJ37ZLB8R': ['B00W259T7G',
  'B013XKHA4M',
  'B0195R1FT8',
  'B01DLR9IDI',
  'B0168SXRR0',
  'B00X4DKZKU',
  'B00Q794RMA',
  'B01ENS6XDC',
  'B01C39X6TW',
  'B0189NOD4I'],
 'A113UOOLBSZN52': ['B00W259T7G',
  'B013XKHA4M',
  'B0195R1FT8',
  'B01DLR9IDI',
  'B0168SXRR0',
  'B00X

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.05593220338983051